In [1]:
import pickle
import h5py
import os
import torch
import random
import torch.utils.data as data
import torch.nn as nn
from torch.autograd import Variable
import numpy


/home/swjung/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class DEAP_DataSet(data.Dataset):
    
    VALID_SPLIT = ('train', 'validation', 'test')
    DATASET = 'data/'
    TEST_TARGET = 1
    TEST_DATA_TYPE = 0
    
    def __init__(self, test_target, test_type=0, split='train', data_dir='processed_data.pkl', label_dir='y_train.pkl'):
        super(DEAP_DataSet, self).__init__()
        
        if split not in self.VALID_SPLIT:
            raise ValueError('Unknown split {:s}'.format(split))
        if not os.path.exists(self.DATASET + data_dir):
            raise ValueError('{:s} does not exist'.format(data_dir))
        if not os.path.exists(self.DATASET + label_dir):
            raise ValueError('{:s} does not exist'.format(label_dir))
            
        data_path = '{}/{}'.format(self.DATASET, data_dir)
        label_path = '{}/{}'.format(self.DATASET, label_dir)
        
        self.split = split
        self.data = pickle.load(open(data_path, 'rb'))
        self.labels = pickle.load(open(label_path, 'rb'))
        
        self.test_target = test_target
        self.test_type = test_type
        self.target_data = []
        
        # create train data set -> 31 * 40 * 32 = 39680
        if self.split == self.VALID_SPLIT[0]:
            counter = 0
            for pp, pp_data in enumerate(self.data):
                if self.test_target is not pp:
                    for c, data in enumerate(pp_data):
                        counter += 1
                        temp_label = self.labels[pp][c][self.test_type]
                        temp_data = data.flatten()
                        self.target_data.append((temp_label, temp_data))
            print(counter)    
        # craete test data set -> 32 * 40 = 1280
        if self.split == self.VALID_SPLIT[2]:
            counter = 0
            for pp, pp_data in enumerate(self.data):
                if self.test_target == pp:
                    for c, data in enumerate(pp_data):
                        counter += 1
                        temp_label = self.labels[pp][c][self.test_type]
                        temp_data = data.flatten()
                        self.target_data.append((temp_label, temp_data))
            print(counter)
                    
    def __getitem__(self, index):
        return self.target_data[index]
    
    def __len__(self):
        return len(self.target_data)

In [3]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(4040, 5000),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(5000, 500),
            nn.ReLU(),
            nn.Dropout(0.50),
            nn.Linear(500, 1000),
            nn.ReLU(),
            nn.Dropout(0.50),
            nn.Linear(1000, 2),
            nn.Dropout(0.50),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        out = self.layer(x)
        return out
    
model = DNN().double().cuda()

In [4]:
# Set HyperParameter
learning_rate = 0.00001
gradient_direction = 0.9
batch_size = 310
epochs = 250

In [5]:
# Load DataSet
DEAP_train = DEAP_DataSet(0, test_type=1)
DEAP_test = DEAP_DataSet(0, test_type=1)

1240
1240


In [6]:
train_loader = torch.utils.data.DataLoader(DEAP_train, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)
test_loader = torch.utils.data.DataLoader(DEAP_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)

In [7]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=gradient_direction)
loss_func = nn.CrossEntropyLoss().double()

In [8]:
for epoch in range(epochs):
    count = 0
    for label, data in train_loader:
        x = Variable(data).double().cuda()
        y_ = Variable(label).type(torch.LongTensor).cuda()

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_).double()
        loss.backward()
        optimizer.step()

        count += 1
        
        if count % 4 == 0 and epoch % 10 == 0:
            print('[{}:{}] loss : {}'.format(epoch, count, loss))

[0:4] loss : 0.7837250491254548
[10:4] loss : 0.7792783958447415
[20:4] loss : 0.753230314933355
[30:4] loss : 0.8002283087365547
[40:4] loss : 0.7731019600586536
[50:4] loss : 0.8026127510024357
[60:4] loss : 0.7809359364971165
[70:4] loss : 0.7695902947500723
[80:4] loss : 0.7741501121344022
[90:4] loss : 0.732852411965116
[100:4] loss : 0.7553741887589764
[110:4] loss : 0.7258830906167377
[120:4] loss : 0.7504687884492035
[130:4] loss : 0.7332483776575924
[140:4] loss : 0.724848958766549
[150:4] loss : 0.7371753745941979
[160:4] loss : 0.7321998549854596
[170:4] loss : 0.7067194668797019
[180:4] loss : 0.727314168791463
[190:4] loss : 0.7017450113330805
[200:4] loss : 0.7194256900419297
[210:4] loss : 0.7529285917550183
[220:4] loss : 0.7197699507760458
[230:4] loss : 0.7185231986706563
[240:4] loss : 0.727098450841872


In [9]:
correct = 0
total = 0

for label, data in test_loader:
    x = Variable(data, requires_grad=False).double().cuda()
    y_ = Variable(label).type(torch.LongTensor).cuda()

    with torch.no_grad():
        output = model.forward(x)
        _, output_index = torch.max(output, 1)

        total += label.size(0)
        correct += (output_index == y_).sum().float()

print('Total : {}, correct : {}'.format(total, correct))
print('Accuracy : {}'.format(100 * correct / total))

Total : 1240, correct : 658.0
Accuracy : 53.06451416015625
